<a href="https://colab.research.google.com/github/MrHidr/AUT.CDM.matrix-accelerated-fpm/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 >**Project
information**  =>  **SN**: *404112060*  |  **SName**: *Mahdi Heydari*  |  CDM Course | Project 1 | Oct 2025

# A Survey of Matrix Monitoring Methods for Accelerating Frequent Pattern Mining Algorithms | First project of Computational Data Mining Course

## summary
[[will be added]]

# Step1: Fetching & Cleaning Data

### Installing Required Packages
ucimlrepo package : fetching dataset from UCI (UC Irvine Machine Learning Repository)

pandas : dataframe control


In [27]:
!pip install ucimlrepo

In [28]:
#importing packages according to "all steps"
from ucimlrepo import fetch_ucirepo
import pandas as pd
import numpy as np
import time

### Fetching Online Retail Dataset from UCI
Documentation of UCI repo for fetch and using dataset: https://github.com/uci-ml-repo/ucimlrepo

In [29]:
online_retail = fetch_ucirepo(id=352)

In [30]:
# data (as pandas dataframes)
X = online_retail.data.features
I = online_retail.data.ids
df = pd.concat([I, X], axis=1)
print("fetched dataset 5 rows as sample:\n", df.head(5))
print("\n","="*30,"\n")
df.info()
print("dataset info:\n", df.describe())

fetched dataset 5 rows as sample:
   InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

      InvoiceDate  UnitPrice  CustomerID         Country  
0  12/1/2010 8:26       2.55     17850.0  United Kingdom  
1  12/1/2010 8:26       3.39     17850.0  United Kingdom  
2  12/1/2010 8:26       2.75     17850.0  United Kingdom  
3  12/1/2010 8:26       3.39     17850.0  United Kingdom  
4  12/1/2010 8:26       3.39     17850.0  United Kingdom  


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       -----

In [31]:
initial_rows = df.shape[0]
print("Initial row count before any cleaning:", initial_rows)

Initial row count before any: 541909


In [32]:
# Exploring Data
# Filter for rows where Quantity is 0 (just to Experiment)
# Just generated to know more about zero quantity datas and what are they look like realy:)
zero_quantity_rows = df[df['Description'] == "returned"]

# Print the 'Description' and other useful columns for these rows
print(zero_quantity_rows[['Quantity', 'Description', 'UnitPrice', 'CustomerID']])

        Quantity Description  UnitPrice  CustomerID
166646         2    returned        0.0         NaN
166647         2    returned        0.0         NaN


### Cleaning Data; Removing Incomplete Records with Missing columns
According to dataset info, Some rows has missing data on "Description" and/or "CustomerID" Columns, as first part of cleaning, we will remove these rows.

In [52]:
# removing datas with Null/Nan values
df_cleaned = df.dropna(subset=['CustomerID', 'Description'])

output_rows_count = df_cleaned.shape[0]
rows_removed = initial_rows - output_rows_count


print("row count after removing rows: ", output_rows_count)
print("removed row count: ", rows_removed)

row count after removing rows:  406829
removed row count:  135080


more info maybe not useful :

we had 4068290 rows with no CustomerID, which is equal to our rows after removing rows with no CustomerID and/or no Description so as result it means if a row has no Description it will have no CustomerID too.

###Cleaning Data; Negetive Values (& Returned Invoices)
While min value of 'Quantity' and 'UnitPrice' are -80995.000000 and -11062.060000, we have rows with negetive quantity and price which make these rows are invalid.
as mentioned in brief we should remove return invoices too, which is covered by removing rows with negetive "Quantity" and as more overthinking, in think of deleting positive quantity invoices according to each negetive quantities, the goal of Not loosing data of selected baskets and not being a financial prefossional but a data oriented thinking :) we just remove negetive baskets.

In [53]:
# Invoice with negative Quantity == Return Invoice
df_cleaned = df_cleaned[df_cleaned['Quantity'] > 0]
# Negative Unit Price (invalid - not a buying basket whatever it is)
df_cleaned = df_cleaned[df_cleaned['UnitPrice'] >= 0]

cleaned_rows_count = df_cleaned.shape[0]
print("row count after removing rows with neg values: ", cleaned_rows_count)
print("removed row count (Neg Price and/or Neg Quantity): ", output_rows_count - cleaned_rows_count)

row count after removing rows with neg values:  397924
removed row count (Neg Price and/or Neg Quantity):  8905


# Step2: Transaction/Item Matrix

## Count Matrix
rows='InvoiceNo'

columns='StockCode count'

In [59]:
count_matrix = df_cleaned.pivot_table(index='InvoiceNo',
                                      columns='StockCode',
                                      aggfunc='size' ,
                                      fill_value=0)

In [60]:
print("5 first rows of Transaction/Item_Count Matrix: \n")
print(count_matrix.head(5))
print("="*30)
print("number of unique InvoiceNo: ", count_matrix.shape[0] ,"\nnumber of unique StockCode", count_matrix.shape[1])

5 first rows of Transaction/Item_Count Matrix: 

StockCode  10002  10080  10120  10123C  10124A  10124G  10125  10133  10135  \
InvoiceNo                                                                     
536365         0      0      0       0       0       0      0      0      0   
536366         0      0      0       0       0       0      0      0      0   
536367         0      0      0       0       0       0      0      0      0   
536368         0      0      0       0       0       0      0      0      0   
536369         0      0      0       0       0       0      0      0      0   

StockCode  11001  ...  90214V  90214W  90214Y  90214Z  BANK CHARGES  C2  DOT  \
InvoiceNo         ...                                                          
536365         0  ...       0       0       0       0             0   0    0   
536366         0  ...       0       0       0       0             0   0    0   
536367         0  ...       0       0       0       0             0   0    0 

## Binary Matrix

In [67]:
binary_matrix = (count_matrix > 0).astype(int)
print(binary_matrix.head(10))
print("Binary Matrix of items (columns) per (transaction) created.")
print("Size of Matrix (same as Count Matrix as expected ): ", binary_matrix.shape)

StockCode  10002  10080  10120  10123C  10124A  10124G  10125  10133  10135  \
InvoiceNo                                                                     
536365         0      0      0       0       0       0      0      0      0   
536366         0      0      0       0       0       0      0      0      0   
536367         0      0      0       0       0       0      0      0      0   
536368         0      0      0       0       0       0      0      0      0   
536369         0      0      0       0       0       0      0      0      0   
536370         1      0      0       0       0       0      0      0      0   
536371         0      0      0       0       0       0      0      0      0   
536372         0      0      0       0       0       0      0      0      0   
536373         0      0      0       0       0       0      0      0      0   
536374         0      0      0       0       0       0      0      0      0   

StockCode  11001  ...  90214V  90214W  90214Y  9021

as we see each row stands for one InvoiceNo and each column show apearance of stock in InvoiceNo (Basket).

# Step3: Streaming Simulation

### InvoiceDate convert to datetime objects

In [73]:
import pandas as pd
import numpy as np
print("Before convertion (3 first): \n",df_cleaned['InvoiceDate'][:3])
df_cleaned['InvoiceDate'] = pd.to_datetime(df_cleaned['InvoiceDate'])
print("="*30)
print("\nAfter convertion (3 first): \n",df_cleaned['InvoiceDate'][:3])
# This code added while in next steps I got to problem for Time object

Before convertion (3 first): 
 0   2010-12-01 08:26:00
1   2010-12-01 08:26:00
2   2010-12-01 08:26:00
Name: InvoiceDate, dtype: datetime64[ns]

After convertion (3 first): 
 0   2010-12-01 08:26:00
1   2010-12-01 08:26:00
2   2010-12-01 08:26:00
Name: InvoiceDate, dtype: datetime64[ns]


## Deviding Batches
In first try using a fix number of batches and seperate datas to N batches with same size (number of rows).
(another aproach will be seperating by Time (for example each 2 months) and in furthor this method need an experiment and correcting to have batches with close size.

In [82]:

sorted_invoice_index = df_cleaned.groupby('InvoiceNo')['InvoiceDate'].min().sort_values().index

# Re-order the main matrix based on the sorted invoice index
sorted_matrix = binary_matrix.loc[sorted_invoice_index]

# Split the sorted matrix into 10 sequential batches
N_BATCHES = 20 #TODO : why 10? what is the best number?
data_stream_batches = np.array_split(sorted_matrix, N_BATCHES)

print(f"Data stream created with {len(data_stream_batches)} batches.")

/usr/local/lib/python3.12/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Data stream created with 20 batches.


## Batches info


In [83]:
min_duration = [200,-1] # [duration,batch_number]
max_duration = [0,-1] # [duration,batch_number] 200 is big enough according to data
for i, batch_df in enumerate(data_stream_batches):

    # Get the list of InvoiceNo's for this batch (they are the index)
    batch_invoice_ids = batch_df.index

    # Filter df_cleaned to get all dates for just those invoices
    relevant_dates = df_cleaned[df_cleaned['InvoiceNo'].isin(batch_invoice_ids)]['InvoiceDate']
    Duration_days = (relevant_dates.max() - relevant_dates.min()).days

    if Duration_days < min_duration[0]:
        min_duration[0] = Duration_days
        min_duration[1] = i+1
    if Duration_days > max_duration[0]:
        max_duration[0] = Duration_days
        max_duration[1] = i+1

    print(f"\n--- Batch {i+1} ---")
    print(f"  Shape (Invoices, Items):", batch_df.shape)
    print(f"  Start Date: ",relevant_dates.min().strftime('%Y-%m-%d'))
    print(f"  End Date: ", relevant_dates.max().strftime('%Y-%m-%d'))
    print(f"  Duration: ",Duration_days," days")
print("="*50)
print("Min Duration: Batch ",min_duration[1]," with", min_duration[0] ," days " )
print("Max Duration: Batch ",max_duration[1]," with", max_duration[0] ," days " )


--- Batch 1 ---
  Shape (Invoices, Items): (927, 3665)
  Start Date:  2010-12-01
  End Date:  2010-12-13
  Duration:  12  days

--- Batch 2 ---
  Shape (Invoices, Items): (927, 3665)
  Start Date:  2010-12-13
  End Date:  2011-01-16
  Duration:  34  days

--- Batch 3 ---
  Shape (Invoices, Items): (927, 3665)
  Start Date:  2011-01-16
  End Date:  2011-02-13
  Duration:  27  days

--- Batch 4 ---
  Shape (Invoices, Items): (927, 3665)
  Start Date:  2011-02-13
  End Date:  2011-03-09
  Duration:  23  days

--- Batch 5 ---
  Shape (Invoices, Items): (927, 3665)
  Start Date:  2011-03-09
  End Date:  2011-03-30
  Duration:  21  days

--- Batch 6 ---
  Shape (Invoices, Items): (927, 3665)
  Start Date:  2011-03-30
  End Date:  2011-04-20
  Duration:  20  days

--- Batch 7 ---
  Shape (Invoices, Items): (927, 3665)
  Start Date:  2011-04-20
  End Date:  2011-05-12
  Duration:  22  days

--- Batch 8 ---
  Shape (Invoices, Items): (927, 3665)
  Start Date:  2011-05-12
  End Date:  2011-06-0

# Step4: Matrix Sketching Algorithms
[[summary of Sketching Goals and reasons]]

## Gaussian Random Projection
`R` is the random projection matrix in GRP algorithm, a matrix of size `d x k` where `d` is the number of unique StockCode (items) and `k` is the target lower dimension which set to `100` for first try.
Note: I'm scaling by $1/\sqrt{k}$ because the theory says this scaling (along with a $1/\sqrt{k}$ factor on the projected data, or just scaling $R$ like this) helps $R$ be "approximately orthogonal" and preserve Euclidean distances. I should double-check this scaling factor in the literature.

In [44]:
k=100
n, d = data_stream_batches[0].shape
print("First batch size:")
print("n=",n,"& d=",d)
R = np.random.normal(0, 1/np.sqrt(k), (d, k))
print("="*30)
print("Random Matrix : ")
print(R)

First batch size:
n= 1854 & d= 3665
Random Matrix : 
[[ 2.05363430e-02  1.12162405e-01  2.99120605e-02 ... -1.01672645e-01
   1.28987755e-01  1.08916956e-01]
 [ 3.88688369e-02  1.77411799e-04  1.11109646e-01 ... -1.43620125e-01
   1.89815763e-01  3.44093516e-02]
 [-6.15308212e-02  1.45199475e-01 -1.71654668e-01 ...  6.14198214e-02
   7.19154803e-02 -1.29005284e-01]
 ...
 [-2.62314966e-02 -1.23706438e-01  1.48831846e-01 ...  9.47592560e-02
   2.08674447e-02  3.10363549e-02]
 [ 6.27896366e-02  9.60766258e-02  1.13925323e-01 ...  4.85450201e-02
   9.03590470e-02 -3.94298238e-02]
 [-5.16132667e-02 -1.38919562e-02  2.63934069e-03 ... -6.65468606e-02
   1.87153136e-01 -2.61063745e-02]]


In [45]:
grp_batch_sketches = []
grp_time_records = []

In [46]:
start_total_time = time.time()

for batch_df in data_stream_batches:

    A_batch = batch_df.values

    batch_start_time = time.time()

    B_sketch = A_batch @ R

    batch_end_time = time.time()

    grp_batch_sketches.append(B_sketch)
    grp_time_records.append(batch_end_time - batch_start_time)

end_total_time = time.time()

print("Whole process took:", end_total_time - start_total_time, "seconds")
print("Average time per batch:", np.mean(grp_time_records), "seconds")
print("max time per batch:", np.max(grp_time_records), "seconds")
print("min time per batch:", np.min(grp_time_records), "seconds")

Whole process took: 2.3198330402374268 seconds
Average time per batch: 0.23171746730804443 seconds
max time per batch: 0.31688928604125977 seconds
min time per batch: 0.1903519630432129 seconds
